# Implementation of Viterbi HMM POS tagger
author: Kailin Zheng<br>
date: October 1 2019<br>
class: HW3, NLP<br>

In [1]:
import nltk 
from nltk.tokenize import word_tokenize

In [2]:
import pandas as pd
import numpy as np

In [6]:
#read Development file (WSJ_24.pos)

#a list of sentence - each line has two lists: word and pos
sentences = [] 

word = []
pos = []


f= open("WSJ_24.pos","r+")


for line in f.readlines():
    words = line.split()
    
    if (words == []):
        sentences.append([word, pos])
        word = []
        pos = []
    else:
        word.append(words[0])
        pos.append(words[1])
f.close()  

# f = open ("WSJ_02-21.pos-chunk","r+")

# for line in f.readlines():
#     words = line.split()
    
#     if (words == []):
#         sentences.append([word, pos])
#         word = []
#         pos = []
#     else:
#         word.append(words[0])
#         pos.append(words[1])
# f.close()


In [ ]:
print(sentences[-1][1][5])

In [7]:
observed_words = []
states = ['start state']
Once = {}
for i in range(len(sentences[:])):
    for j in range(len(sentences[i][0])):
        word = sentences[i][0][j]
        tag = sentences[i][1][j]
        if word in Once.keys():
            del Once[word]
        Once[word] = tag
        
        if (word not in observed_words):
            observed_words.append(word)
        if (tag not in states):
            states.append(tag)
states.append('final state')
print(states)
print(observed_words[:10])

['start state', 'DT', 'NN', 'POS', 'MD', 'VB', 'VBN', 'IN', 'JJ', 'NNS', ',', 'CC', '.', 'RBS', 'NNP', 'VBZ', 'TO', '$', 'CD', 'VBG', 'RB', 'VBD', 'PRP', 'PRP$', 'VBP', 'WRB', '``', 'EX', "''", ':', 'RBR', 'WP', 'JJR', 'WDT', 'RP', '#', 'PDT', 'UH', '-LRB-', '-RRB-', 'NNPS', 'JJS', 'SYM', 'FW', 'WP$', 'LS', 'final state']
['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken', 'from', 'several', 'vantage']


In [8]:
OOV = pd.Series(0.0, index = states)
for v in Once.keys():
    OOV[Once[v]] += 1
    
OOV = OOV/len(Once)

In [9]:
#A = N ⨯ N matrix such that a_i,j is the probability of the transition from q_i to q_j
#B = lookup table such that bi(wt) is the probability that POS i is realized as word t
#q0 = start state, q1...qN, qf = final state

A = pd.DataFrame(0.0, index = states, columns = states)
B = pd.DataFrame(0.0, index = observed_words, columns = states)


for i in range(len(sentences[:])):
    prev = 'start state'
    for j in range(len(sentences[i][1])):
        now = sentences[i][1][j]
        A.loc[prev,now] += 1
        prev = now
        
        B.loc[sentences[i][0][j], sentences[i][1][j]] += 1

A = A.div(A.sum(axis=1), axis=0)
B = B.div(B.sum(axis=1), axis=0)
#B

In [ ]:
#A

In [12]:
#read test file (WSJ_23.pos)

#a list of sentence - each line has two lists: word and pos
S = [] 

word = []
pos = []
f= open("test_words.txt","r+")

for line in f.readlines():
    words = line.split()
    
    if (words == []):
        S.append([word, pos])
        word = []
        pos = []
    else:
        word.append(words[0])
    
print(S[1])

[['Cover', ',', 'and', 'cook', 'for', '5', 'to', '6', 'hours', 'on', 'High', '.'], []]


In [13]:
def viterbi_algo(A,B,S):
    result = []
    for i in range(len(S)):
        if i % 50 == 0:
            print(i)
        
        

        viterbi = pd.DataFrame(0.0, index = states, columns = S[i][0])
        viterbi.insert(0, column = '\\start', value = np.zeros(len(states)))
        viterbi.insert(len(viterbi.columns), column = '\\end', value = 0.0)
        viterbi.loc['start state', '\\start'] = 1
        backpointer = pd.Series(-1, index = viterbi.columns)
        
        #initialize all states of the first word in viterbi: aka column[1]
        for q in range(1, len(states)):
            state = states[q]
            first_word = S[i][0][0]
            if (first_word not in observed_words):
                viterbi.iloc[q,1] = A.iloc[0,q] * OOV[states[q]]
            else:
                viterbi.iloc[q,1] = A.iloc[0,q] * B.loc[first_word, state]
            backpointer[1] = 0
        #print(viterbi)
        
        #w in viterbi, starting 1 = w-1 in sentence, starting 0
        for w in range(1, len(viterbi.columns)-1):
            #print(i,',',w)
            word = S[i][0][w-1]
            score = 0
            for q in range(len(states)): #state of prev word
                if viterbi.iloc[q][w-1] == 0:
                    continue
                for p in range(len(states)): #state of curr word
                    curr_state = states[p]
                    if word not in observed_words:
                        score = viterbi.iloc[q, w-1] * A.iloc[q,p] * OOV[states[q]]
                    else:
                        if B.loc[word, curr_state] == 0:
                            continue
                        score = viterbi.iloc[q, w-1] * A.iloc[q,p] * B.loc[word, curr_state]
                    if score > viterbi.iloc[p,w]:
                        viterbi.iloc[p,w] = score
                        backpointer[w] = q
        

        max_index = -1
        for q in range(len(states)):
            if viterbi.iloc[q,-2] > viterbi.iloc[-1,-1]:
                max_index = q
                viterbi.iloc[-1,-1] = viterbi.iloc[q,-2]
                backpointer[-1] = q
                

        #print(viterbi)
        #print(backpointer)
        result.append(backpointer)
    return result

In [14]:
backpointers = viterbi_algo(A,B,S)

0
50
100
150
200
250
300
350
400
450


In [15]:
result = open("test.pos","w+")
for i in range(len(S)):

    b = backpointers[i]
    for w in range(len(S[i][0])):
        result.write(S[i][0][w]+'\t'+states[b[w+2]])
        result.write('\n')
    result.write('\n')
result.close()

In [2]:
count = len(open("test.pos").readlines(  ))


In [3]:
count

6587